In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as skTrans
from google.colab import files
import altair as alt


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:
upload= files.upload() #choose 'ADNI_Features_1-3.csv'

Saving ADNI_Features_1-3v4.csv to ADNI_Features_1-3v4.csv


In [ ]:
#Choose only ones not used for training

ADNI_data = pd.read_csv('ADNI_Features_1-3v4.csv')
df = ADNI_data[ADNI_data['source']!='train']
df

,scan_ID,Diagnosis,source,SubjectID,SessionId,DiagnosisVisDate,UniqueID,Feature_1,Feature_2,Feature_3,...,Feature_1015,Feature_1016,Feature_1017,Feature_1018,Feature_1019,Feature_1020,Feature_1021,Feature_1022,Feature_1023,Feature_1024
0,031S0351,2.0,unseen,031_S_0351,2006-11-02,2006-11-07,I28618,0.060345,-0.310687,-0.155691,...,-0.408503,0.370602,0.169428,-0.157290,-0.375038,-0.315630,-0.295684,-0.097642,-0.162235,0.719451
1,031S0351,2.0,unseen,031_S_0351,2008-04-21,2008-04-29,I103450,0.098181,-0.285875,0.096466,...,-0.676731,0.449639,0.043787,-0.221529,-0.367277,-0.499006,-0.400847,-0.055737,-0.162698,0.882688
14,136S0695,2.0,val,136_S_0695,2007-09-17,2007-09-19,I73403,0.030187,-0.478979,-0.152521,...,-0.487125,0.243526,0.046530,0.049963,0.278593,0.050035,-0.378073,0.152009,-0.251996,0.715327
15,136S0695,3.0,val,136_S_0695,2008-09-22,2008-09-25,I118266,0.088016,-0.240342,-0.215628,...,-0.319857,0.373594,0.463427,-0.161745,0.146263,0.098585,-0.497146,0.064602,-0.107501,0.458334
16,136S0695,2.0,val,136_S_0695,2006-09-18,2006-09-25,I24585,0.202961,-0.204472,-0.009518,...,-0.599153,0.411147,0.350229,-0.106523,-0.114869,0.018005,-0.499099,-0.062719,-0.304241,0.446574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,014S0548,1.0,val,014_S_0548,2006-05-25,2006-06-08,I16024,-0.242432,-0.010864,0.121730,...,-0.109987,0.316109,0.231225,-0.361799,0.079556,-0.208299,-0.209442,0.307932,0.024379,0.571480
518,005S1341,3.0,test,005_S_1341,2007-10-02,2007-10-01,I76567,-0.370205,-0.667201,0.076087,...,-0.316471,0.403319,-0.244787,0.034111,-0.093803,-0.064739,-0.239481,0.444897,-0.319565,0.503523
519,005S1341,3.0,test,005_S_1341,2007-03-07,2007-04-04,I43188,-0.382090,-0.579963,0.084748,...,-0.271949,0.464197,-0.016319,0.121341,0.092344,0.129815,-0.237085,0.385299,-0.188575,0.332562
520,005S1341,3.0,test,005_S_1341,2009-04-03,2009-04-27,I140378,-0.246994,-0.459477,0.289620,...,-0.417529,0.278192,-0.160744,-0.238909,-0.075424,0.091035,-0.395902,0.422412,-0.267931,0.598010


In [ ]:
# Don't want non-numeric columns: leave only diagnosis and features
numeric_df = df.select_dtypes(include=[float, int]).dropna()
numeric_df

,Diagnosis,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_1015,Feature_1016,Feature_1017,Feature_1018,Feature_1019,Feature_1020,Feature_1021,Feature_1022,Feature_1023,Feature_1024
0,2.0,0.060345,-0.310687,-0.155691,0.045950,-0.728961,-0.424921,-0.285582,0.310577,-0.493605,...,-0.408503,0.370602,0.169428,-0.157290,-0.375038,-0.315630,-0.295684,-0.097642,-0.162235,0.719451
1,2.0,0.098181,-0.285875,0.096466,0.181360,-0.843716,-0.369934,-0.175737,0.183176,-0.627998,...,-0.676731,0.449639,0.043787,-0.221529,-0.367277,-0.499006,-0.400847,-0.055737,-0.162698,0.882688
14,2.0,0.030187,-0.478979,-0.152521,0.556756,-0.586834,-0.415396,-0.005562,-0.010656,-0.561178,...,-0.487125,0.243526,0.046530,0.049963,0.278593,0.050035,-0.378073,0.152009,-0.251996,0.715327
15,3.0,0.088016,-0.240342,-0.215628,0.488431,-0.437098,-0.529650,-0.295986,-0.001217,-0.412511,...,-0.319857,0.373594,0.463427,-0.161745,0.146263,0.098585,-0.497146,0.064602,-0.107501,0.458334
16,2.0,0.202961,-0.204472,-0.009518,0.273330,-0.502865,-0.514745,-0.502081,-0.069701,-0.504464,...,-0.599153,0.411147,0.350229,-0.106523,-0.114869,0.018005,-0.499099,-0.062719,-0.304241,0.446574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,1.0,-0.242432,-0.010864,0.121730,-0.238080,-0.147753,-0.025475,0.013753,0.239555,-0.603327,...,-0.109987,0.316109,0.231225,-0.361799,0.079556,-0.208299,-0.209442,0.307932,0.024379,0.571480
518,3.0,-0.370205,-0.667201,0.076087,0.235904,-0.672224,-0.497349,-0.327367,0.157322,-0.096926,...,-0.316471,0.403319,-0.244787,0.034111,-0.093803,-0.064739,-0.239481,0.444897,-0.319565,0.503523
519,3.0,-0.382090,-0.579963,0.084748,0.318212,-0.830009,-0.374211,-0.137538,0.104542,-0.143115,...,-0.271949,0.464197,-0.016319,0.121341,0.092344,0.129815,-0.237085,0.385299,-0.188575,0.332562
520,3.0,-0.246994,-0.459477,0.289620,0.482559,-0.601298,-0.472030,-0.251261,0.131945,-0.157444,...,-0.417529,0.278192,-0.160744,-0.238909,-0.075424,0.091035,-0.395902,0.422412,-0.267931,0.598010


##PCA and 3-way Classifier (LinearAlexNet)

In [ ]:
# @title PCA + T-SNE


# Apply PCA
pca = PCA(n_components=0.99)  # Preserve 99% of variance
pca_result = pca.fit_transform(numeric_df)
pca_df = pd.DataFrame(data=pca_result)


# Apply t-SNE
tsne = TSNE(n_components=2, verbose=1, perplexity=5, n_iter=300)
#tsne_results = tsne.fit_transform(pca_result)
tsne_results = tsne.fit_transform(numeric_df)

# Combine t-SNE results with the original df
tsne_df = pd.DataFrame(data=tsne_results, columns=["t-SNE1", "t-SNE2"])
result_df = pd.concat([ADNI_data['Diagnosis'], tsne_df], axis=1).reset_index(drop=True)

scatter_plot = alt.Chart(result_df).mark_circle(size=60).encode(
    x='t-SNE1',
    y='t-SNE2',
    color='Diagnosis:N'
).properties(
    width=400,
    height=400,
    title='t-SNE visualization of PCA-reduced data'
)

scatter_plot

[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 238 samples in 0.002s...
[t-SNE] Computed neighbors for 238 samples in 0.042s...
[t-SNE] Computed conditional probabilities for sample 238 / 238
[t-SNE] Mean sigma: 1.516245
[t-SNE] KL divergence after 250 iterations with early exaggeration: 72.346954
[t-SNE] KL divergence after 300 iterations: 1.076800


alt.Chart(...)

In [ ]:
# @title Import LinearClassifierAlexNet
#need to import the model
model_upload = files.upload() #choose classifier.py
from classifier import LinearClassifierAlexNet


Saving classifier.py to classifier.py


In [ ]:
# @title Apply PCA
numeric_with_diagnosis_df = df.select_dtypes(include=[float, int])
numeric_with_diagnosis_df['Diagnosis'] = df['Diagnosis']

#Drop rows with any NaN values
numeric_with_diagnosis_df.dropna(inplace=True)

# Separate Diagnoses
diagnosis = numeric_with_diagnosis_df['Diagnosis']
numeric_df = numeric_with_diagnosis_df.drop(columns=['Diagnosis'])

# Apply PCA
pca = PCA(n_components=0.999)  # Preserve 99.9% of variance
pca_result = pca.fit_transform(numeric_df)

pca_df = pd.DataFrame(data=pca_result)

# Concatenate Diagnosis column back
pca_df = pd.concat([diagnosis.reset_index(drop=True), pca_df.reset_index(drop=True)], axis=1)
pca_df


,Diagnosis,0,1,2,3,4,5,6,7,8,...,221,222,223,224,225,226,227,228,229,230
0,2.0,0.715261,-0.332469,-1.915164,0.936576,1.877976,-0.571068,-0.435435,-0.030196,-0.037045,...,-0.090846,0.132208,-0.309025,0.127181,-0.026341,0.231964,-0.276790,0.173677,0.279645,0.186482
1,2.0,1.353052,-0.411292,-1.909783,0.673223,1.765788,-0.452149,-0.600386,-0.082489,-0.003550,...,0.120964,-0.140667,0.273646,-0.188784,-0.086130,-0.271950,0.314483,-0.121651,-0.253897,-0.127651
2,2.0,-0.523199,1.315941,2.533163,-0.161813,0.042967,-0.213433,-1.582311,0.521314,-0.417360,...,0.102136,-0.055995,-0.084895,0.164318,-0.191853,0.059178,-0.006147,0.073620,-0.012355,-0.070530
3,3.0,-1.562067,-0.203069,1.477449,0.506067,0.807273,0.200438,-1.412742,-0.173163,-0.837118,...,-0.091060,-0.001800,-0.179046,0.098166,0.060339,-0.087491,-0.085579,0.018030,-0.164781,-0.045125
4,2.0,-1.931265,-1.310138,1.286539,0.161028,0.972368,0.801144,-1.539378,-1.043964,-0.295623,...,0.146551,0.028600,0.283691,-0.010928,-0.100872,0.005089,-0.075176,-0.085654,0.211246,-0.036239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,1.0,-2.941595,3.823022,0.380056,1.246894,0.893827,1.378430,1.352518,0.813210,1.898040,...,0.044376,0.071284,0.089913,0.029532,-0.026971,0.059642,0.019284,-0.083111,0.047130,-0.032052
234,3.0,2.302722,2.064969,3.276757,1.342350,-0.223755,-0.134806,0.542878,-0.175634,0.221332,...,0.042520,-0.036107,-0.073386,-0.126494,-0.070501,-0.076538,-0.040608,0.113321,0.094151,-0.062660
235,3.0,1.242236,2.546366,2.640848,1.061489,-0.855281,-0.355733,0.455460,-0.464469,-0.309073,...,-0.049070,-0.007997,0.074334,0.065096,0.005215,0.028896,0.118809,-0.029766,-0.070487,0.034463
236,3.0,2.121497,1.361986,3.194428,1.276124,-0.608195,0.629025,1.532160,-1.135729,0.242120,...,-0.091054,0.068756,0.099311,0.117266,0.067700,0.098812,0.002656,-0.023843,-0.059314,0.103228


In [ ]:
# @title Apply AlexNet Linear Classifier
#Apply their AlexNet Classifier:

pca_df = pca_df.dropna()

#Select features as X and  diagnosis as target
X = pca_df.drop(columns=['Diagnosis'])#,'scan_ID',  'source', 'SubjectID', 'SessionId'])
y = pca_df['Diagnosis']

# Encode the categorical target variable from 1,2,3 to 0,1,2
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert to pyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)



#Initialize the classifier
in_dim = X_train.shape[1]
model = LinearClassifierAlexNet(in_dim)

# train the model
criterion = torch.nn.CrossEntropyLoss() #loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) #optimizer

#Training loop
epochsmax = 30
epochs = np.arange(0,epochsmax,1)
accuracy = []
counter = 0

for num_epochs in range(len(epochs)):

  for epoch in epochs:
    counter +=1
    #eval (forward pass)
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    #Backwards and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if counter % 1000 == 0: print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}') #to minimize printing

  # Test the model
  model.eval()

  with torch.no_grad():
    correct = 0
    total = 0
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    total += y_test.size(0)
    correct += (predicted == y_test).sum().item()
  accuracy.append(correct / total * 100)

print('Accuracy of the model on the test set: {} %'.format(100 * correct / total))

In [ ]:
AlexNetClassifier = pd.DataFrame({'Epochs': epochs, 'Accuracy': accuracy, 'Learning Rate': 0.001})
AlexNetClassifier

,Epochs,Accuracy,Learning Rate
0,0,43.750000,0.001
1,1,39.583333,0.001
2,2,37.500000,0.001
3,3,37.500000,0.001
4,4,41.666667,0.001
5,5,39.583333,0.001
6,6,37.500000,0.001
7,7,37.500000,0.001
8,8,37.500000,0.001
9,9,37.500000,0.001


In [ ]:
chart = alt.Chart(AlexNetClassifier).mark_circle().encode(
    x='Epochs',
    y = 'Accuracy'
)
chart

alt.Chart(...)

In [ ]:


# Function to run the model with different parameters
def run_model(X, y_encoded, loss_function, optimizer_function, lr, pca_components):
    # PCA transformation
    pca = PCA(n_components=pca_components)
    X_pca = pca.fit_transform(X)

    # split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y_encoded, test_size=0.2, random_state=42)

    # Convert to tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)

    in_dim = X_train.shape[1]
    model = LinearClassifierAlexNet(in_dim)

    # Loss and optimizer
    criterion = loss_function()
    optimizer = optimizer_function(model.parameters(), lr=lr)

    # Training loop
    epochsmax = 50


    accuracy = []
    for epoch in range(epochsmax):
        # Forward
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Test
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            outputs = model(X_test)
            _, predicted = torch.max(outputs.data, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()
        accuracy.append(correct / total * 100)

    return accuracy

# Load and preprocess the data
pca_df = pca_df.dropna()
X = pca_df.drop(columns=['Diagnosis'])
y = pca_df['Diagnosis']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Parameters to try
loss_functions = [torch.nn.CrossEntropyLoss, torch.nn.NLLLoss]
optimizer_functions = [torch.optim.Adam, torch.optim.SGD]
learning_rates = [0.01, 0.001, 0.0001]
pca_components_list = [0.9, 0.95, 0.99, 0.999]

# Running the model with different parameters
results = []
for loss_function in loss_functions:
    for optimizer_function in optimizer_functions:
        for lr in learning_rates:
            for pca_components in pca_components_list:
                accuracy = run_model(X, y_encoded, loss_function, optimizer_function, lr, pca_components)
                for epoch, acc in enumerate(accuracy):
                    results.append({
                        'Epoch': epoch,
                        'Accuracy': acc,
                        'Loss Function': loss_function.__name__,
                        'Optimizer': optimizer_function.__name__,
                        'Learning Rate': lr,
                        'PCA Components': pca_components
                    })

results_df = pd.DataFrame(results)

chart = alt.Chart(results_df).mark_circle().encode(
    x='Epoch:Q',
    y=alt.Y('Accuracy:Q', scale=alt.Scale(domain=[0, 100])),
    color='Learning Rate:N',
    tooltip=['Epoch', 'Accuracy', 'Loss Function', 'Optimizer', 'Learning Rate', 'PCA Components']
).properties(
    width=180,
    height=150
)

# Create small multiples
chart = chart.facet(
    column='Loss Function:N',
    row='Optimizer:N'
).resolve_scale(
    x='independent',
    y='independent'
).properties(
    title="Linear Classifier Accuracy"
)
chart = chart.configure_title(
    align='center'
)

chart


alt.FacetChart(...)

In [ ]:
# @title LinearClassifierAlexNet


# Function to run the model with different parameters
def run_model(X, y_encoded, loss_function, optimizer_function, lr, pca_components):
    # PCA transformation
    pca = PCA(n_components=pca_components)
    X_pca = pca.fit_transform(X)

    # Splitting the dataset
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y_encoded, test_size=0.2, random_state=42)

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)

    # Initialize the classifier
    in_dim = X_train.shape[1]
    model = LinearClassifierAlexNet(in_dim)

    # Loss and optimizer
    criterion = loss_function()
    optimizer = optimizer_function(model.parameters(), lr=lr)

    # Training loop
    epochsmax = 50
    metrics_dict = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': []
    }


    accuracy = []
    for epoch in range(epochsmax):
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Test the model
        model.eval()
        with torch.no_grad():
            outputs = model(X_test)
            _, predicted = torch.max(outputs.data, 1)
            accuracy = (predicted == y_test).sum().item() / y_test.size(0)
            precision = precision_score(y_test.numpy(), predicted.numpy(), average='macro', zero_division=0)
            recall = recall_score(y_test.numpy(), predicted.numpy(), average='macro')
            f1 = f1_score(y_test.numpy(), predicted.numpy(), average='macro')

            metrics_dict['accuracy'].append(accuracy * 100)
            metrics_dict['precision'].append(precision)
            metrics_dict['recall'].append(recall)
            metrics_dict['f1'].append(f1)

    return metrics_dict

# Load and preprocess the data
pca_df = pca_df.dropna()
X = pca_df.drop(columns=['Diagnosis'])
y = pca_df['Diagnosis']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Parameters to try
loss_functions = [torch.nn.CrossEntropyLoss, torch.nn.NLLLoss]
optimizer_functions = [torch.optim.Adam, torch.optim.SGD]
learning_rates = [0.01, 0.001, 0.0001]
pca_components_list = [0.9, 0.95, 0.99, 0.999]

# Running the model with different parameters
results = []
for loss_function in loss_functions:
    for optimizer_function in optimizer_functions:
        for lr in learning_rates:
            for pca_components in pca_components_list:
                metrics = run_model(X, y_encoded, loss_function, optimizer_function, lr, pca_components)
                for epoch in range(50):  # or epochsmax
                    results.append({
                        'Epoch': epoch,
                        'Accuracy': metrics['accuracy'][epoch],
                        'Precision': metrics['precision'][epoch],
                        'Recall': metrics['recall'][epoch],
                        'F1 Score': metrics['f1'][epoch],
                        'Loss Function': loss_function.__name__,
                        'Optimizer': optimizer_function.__name__,
                        'Learning Rate': lr,
                        'PCA Components': pca_components
                    })

results_df = pd.DataFrame(results)


base = alt.Chart(results_df).encode(
    alt.X('Epoch:Q')
).properties(
    width=180,
    height=150
)

# Define line charts for each metric
accuracy_chart = base.mark_line(color='blue').encode(y='Accuracy:Q')
precision_chart = base.mark_line(color='red').encode(y='Precision:Q')
recall_chart = base.mark_line(color='green').encode(y='Recall:Q')
f1_chart = base.mark_line(color='orange').encode(y='F1 Score:Q')

# Layer the charts
final_chart = alt.layer(accuracy_chart, precision_chart, recall_chart, f1_chart).resolve_scale(
    y='independent'
).properties(
    title="Model Performance Across Epochs"
)

final_chart.configure_title(align='center')



alt.LayerChart(...)

In [ ]:
# @title Metric charts
def create_metric_chart(df, metric_name, color):
    return alt.Chart(df).mark_line(color=color).encode(
        x=alt.X('Epoch:Q', axis=alt.Axis(title='Epoch')),
        y=alt.Y(f'{metric_name}:Q', axis=alt.Axis(title=metric_name)),
        tooltip=['Epoch', metric_name, 'Loss Function', 'Optimizer', 'Learning Rate', 'PCA Components']
    ).properties(
        width=300,
        height=200,
        title=f'{metric_name} Over Epochs'
    )


accuracy_chart = create_metric_chart(results_df, 'Accuracy', 'blue')
precision_chart = create_metric_chart(results_df, 'Precision', 'red')
recall_chart = create_metric_chart(results_df, 'Recall', 'green')
f1_chart = create_metric_chart(results_df, 'F1 Score', 'orange')

accuracy_chart.display()
precision_chart.display()
recall_chart.display()
f1_chart.display()


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

##Two-Way Classifier

In [ ]:
# @title Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA

#Compare CN and AD
CN_AD = df[(df['Diagnosis'] == 1.0)|(df['Diagnosis'] == 3.0)]
CN_AD = CN_AD.replace({3.0:2})

#Split x and y
Y = CN_AD['Diagnosis']
CN_AD = CN_AD.drop(columns = ['Diagnosis','scan_ID','source','SubjectID','SessionId','DiagnosisVisDate','UniqueID'])

#split train and test sets
X_train, X_test, y_train, y_test = train_test_split(CN_AD, Y, test_size=0.2, random_state=42)

n_estimators = [100,200,300,400]
loss = ['exponential','log_loss'] #exponential -> adaboost
learning_rate = [0.0001, 0.001, 0.01,0.1,0.2]
max_depth = [1,2,3,4,5]
n_components = [0.9, 0.95, 0.99, 0.999]
results = []

for p in n_components:
  pca = PCA(n_components = p)
  X_train_pca = pca.fit_transform(X_train)
  X_test_pca = pca.transform(X_test)
  for i in n_estimators:
    for j in loss:
      for k in learning_rate:
        for m in max_depth:
          clf = GradientBoostingClassifier(loss = j, n_estimators = i, learning_rate=k, max_depth = m, random_state = 42).fit(X_train_pca, y_train)
          score=clf.score(X_test_pca, y_test)
          results.append({'n_estimators': i,
                          'loss': j,
                          'learning_rate':k,
                          'max_depth': m,
                          'score':score,
                          'pca_components': p})
results_df = pd.DataFrame(results)


In [ ]:
# @title Gradient Boosting Classifier chart

alt.Chart(results_df).mark_circle().encode(
    x= 'n_estimators',
    y='score',
    tooltip = ['n_estimators','loss','learning_rate','max_depth','score']
)

alt.Chart(...)

In [ ]:
#file uploads if kernel dies
from google.colab import files
model_upload = files.upload() #choose classifier.py, ADNI_Features_1-3v4.csv
upload = files.upload()

Saving classifier.py to classifier.py


Saving ADNI_Features_1-3v4.csv to ADNI_Features_1-3v4.csv


In [ ]:
# @title AUC Calculation
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc
import itertools
import random
import numpy as np
import pandas as pd

import torch.nn.functional as F
from sklearn.decomposition import PCA


#Increase reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

df = pd.read_csv('ADNI_Features_1-3v4.csv').dropna()
df = df[(df['Diagnosis'] == 1.0) | (df['Diagnosis'] == 3.0)] #leave out MCI but left in model training data for now

from classifier import AdversarialClassifier, LinearClassifierAlexNet

Y = df['Diagnosis']
X = df.drop(columns=['Diagnosis', 'scan_ID', 'source', 'SubjectID', 'SessionId', 'DiagnosisVisDate', 'UniqueID'])



X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

#Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)


y_train = y_train.map({1.0: 0, 3.0: 1})
y_val = y_val.map({1.0: 0, 3.0: 1})
y_test = y_test.map({1.0: 0, 3.0: 1})


# Convert to PyTorch tensors
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_val = torch.tensor(y_val.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)


#DataLoader for validation
val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(dataset=val_data, batch_size=64)


#model = LinearClassifierAlexNet(in_dim=X_train.shape[1], n_label=2)
model = AdversarialClassifier(in_dim=X_train.shape[1], nhid=200, out_dim = 2)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)


num_epochs = 100

for epoch in range(num_epochs):
    # Training phase
    model.train()
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backwards + optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation phase
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / len(val_loader.dataset)



# Predict probabilities for the positive class
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    probabilities = torch.nn.functional.softmax(outputs, dim=1)[:, 1]

auc_score = roc_auc_score(y_test.numpy(), probabilities.numpy())
print(f"AUC Score: {auc_score}")

AUC Score: 0.625


In [ ]:
# @title Grid Search Compare Accuracy


from classifier import AdversarialClassifier
from classifier import LinearClassifierAlexNet

# Define parameter grids
param_grid_AdversarialClassifier = {
    'nhid': [100, 200, 300],
    'lr': [0.001, 0.01, 0.1],
    'momentum': [0.8, 0.9, 0.95, 0.99]
}

param_grid_LinearClassifierAlexNet = {
    'n_hid': [100, 200, 300],
    'lr': [0.001, 0.01, 0.1],
    'momentum': [0.8, 0.9, 0.95, 0.99]
}

# Function to train and evaluate a model
def train_evaluate_model(model, train_loader, val_loader, num_epochs, lr, momentum):
    criterion = nn.CrossEntropyLoss()
    #optimizer = optim.Adam(model.parameters(), lr=lr)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / len(val_loader.dataset)
    return val_loss, val_accuracy

# Function to perform grid search
def grid_search(param_grid, classifier_type, train_loader, val_loader, num_epochs):
    best_accuracy = 0
    best_params = {}

    for params in itertools.product(*param_grid.values()):
        param_dict = dict(zip(param_grid.keys(), params))

        if classifier_type == 'AdversarialClassifier':
            model = AdversarialClassifier(in_dim=X_train.shape[1], nhid=param_dict['nhid'], out_dim=param_dict['out_dim'])
        elif classifier_type == 'LinearClassifierAlexNet':
            model = LinearClassifierAlexNet(in_dim=X_train.shape[1], n_hid=param_dict['n_hid'], n_label=param_dict['n_label'])

        val_loss, val_accuracy = train_evaluate_model(model, train_loader, val_loader, num_epochs, param_dict['lr'], param_dict['momentum'])

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_params = param_dict

    return best_params, best_accuracy


train_loader = DataLoader(dataset=TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=TensorDataset(X_val, y_val), batch_size=64)

# Grid search for AdversarialClassifier
best_params_AdversarialClassifier, best_accuracy_AdversarialClassifier = grid_search(
    param_grid_AdversarialClassifier, 'AdversarialClassifier', train_loader, val_loader, 100)

# Grid search for LinearClassifierAlexNet
best_params_LinearClassifierAlexNet, best_accuracy_LinearClassifierAlexNet = grid_search(
    param_grid_LinearClassifierAlexNet, 'LinearClassifierAlexNet', train_loader, val_loader, 100)

print("Best Params for AdversarialClassifier:", best_params_AdversarialClassifier, "with accuracy:", best_accuracy_AdversarialClassifier)
print("Best Params for LinearClassifierAlexNet:", best_params_LinearClassifierAlexNet, "with accuracy:", best_accuracy_LinearClassifierAlexNet)


Best Params for AdversarialClassifier: {'nhid': 300, 'out_dim': 2, 'lr': 0.01, 'momentum': 0.99} with accuracy: 63.63636363636363
Best Params for LinearClassifierAlexNet: {'n_hid': 200, 'n_label': 2, 'lr': 0.001, 'momentum': 0.8} with accuracy: 63.63636363636363


In [ ]:
# @title Grid Search Compare AUC
# @markdown Stochastic Gradient Descent w/ Cross Entropy Loss and batch size of 64. Compares for both LinearClassifierAlexNet and AdversarialClassifier from classifier.py number of hidden layers, learning rates, momentum, number of pca components, and number of epochs.


#Increase reproducibility
seed = 20021102
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

df = pd.read_csv('ADNI_Features_1-3v4.csv').dropna()
df = df[(df['Diagnosis'] == 1.0) | (df['Diagnosis'] == 3.0)] #leave out MCI but left in model training data for now

from classifier import AdversarialClassifier
from classifier import LinearClassifierAlexNet

Y = df['Diagnosis']
X = df.drop(columns=['Diagnosis', 'scan_ID', 'source', 'SubjectID', 'SessionId', 'DiagnosisVisDate', 'UniqueID'])

scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42)

#Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

#map diagnoses
y_train = y_train.map({1.0: 0, 3.0: 1})
y_val = y_val.map({1.0: 0, 3.0: 1})
y_test = y_test.map({1.0: 0, 3.0: 1})


# Convert to PyTorch tensors
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_val = torch.tensor(y_val.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)


#DataLoader for validation
val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(dataset=val_data, batch_size=64)

from classifier import AdversarialClassifier
from classifier import LinearClassifierAlexNet

train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)


# Define parameter grids
param_grid_AdversarialClassifier = {
    'nhid': [100, 200, 300, 400],
    'lr': [0.0001, 0.001, 0.01],
    'momentum': [0.8, 0.9, 0.93, 0.95, 0.99],
    'num_epochs' : [100, 125, 150, 175, 200, 300, 400, 500],
    'pca_components': [0.9, 0.95, 0.99, None]
}

param_grid_LinearClassifierAlexNet = {
    'n_hid': [100, 200, 300, 400],
    'lr': [0.0001, 0.001, 0.01],
    'momentum': [0.8, 0.9, 0.93, 0.95, 0.99],
    'num_epochs' : [100, 125, 150, 175, 200, 300, 400, 500],
    'pca_components': [0.9, 0.95, 0.99, None]
}

def apply_pca(X_train, X_val, n_components):
    if n_components is None:
        return X_train, X_val  # No PCA applied

    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_val_pca = pca.transform(X_val)  # Transform validation data using training PCA fit
    return X_train_pca, X_val_pca


def train_evaluate_model(model, train_loader, val_loader, num_epochs, lr, momentum):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                probabilities = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(probabilities)

                # Check for NaNs in predictions
                if np.isnan(probabilities).any():
                    raise ValueError("NaN found in model predictions")

    if np.isnan(all_labels).any():
        raise ValueError("NaN found in validation labels")

    auc_score = roc_auc_score(all_labels, all_preds)
    return auc_score


def grid_search(param_grid, classifier_type, X_train, y_train, X_val, y_val):
    best_auc = 0
    best_params = {}

    for params in itertools.product(*param_grid.values()):
        param_dict = dict(zip(param_grid.keys(), params))

        # Apply PCA with the number of components set
        X_train_pca, X_val_pca = apply_pca(X_train, X_val, param_dict['pca_components'])

        # Adjust in_dim for the PCA-transformed data
        in_dim = X_train_pca.shape[1]

        # Make Dataloaders for the PCA data
        train_data_pca = TensorDataset(torch.tensor(X_train_pca, dtype=torch.float32), y_train)
        train_loader_pca = DataLoader(dataset=train_data_pca, batch_size=64, shuffle=True)
        val_data_pca = TensorDataset(torch.tensor(X_val_pca, dtype=torch.float32), y_val)
        val_loader_pca = DataLoader(dataset=val_data_pca, batch_size=64)

        # Initialize the model with the adjusted in_dim
        if classifier_type == 'AdversarialClassifier':
            model = AdversarialClassifier(in_dim=in_dim, nhid=param_dict['nhid'], out_dim=2)
        elif classifier_type == 'LinearClassifierAlexNet':
            model = LinearClassifierAlexNet(in_dim=in_dim, n_hid=param_dict['n_hid'], n_label=2)


        # Train and evaluate the model
        auc_score = train_evaluate_model(model, train_loader_pca, val_loader_pca, param_dict['num_epochs'], param_dict['lr'], param_dict['momentum'])
        print(auc_score)


        if auc_score > best_auc:
            best_auc = auc_score
            best_params = param_dict


    return best_params, best_auc

best_params_AdversarialClassifier, best_auc_AdversarialClassifier = grid_search(
    param_grid_AdversarialClassifier, 'AdversarialClassifier', X_train.numpy(), y_train, X_val.numpy(), y_val)

best_params_LinearClassifierAlexNet, best_auc_LinearClassifierAlexNet = grid_search(
    param_grid_LinearClassifierAlexNet, 'LinearClassifierAlexNet', X_train.numpy(), y_train, X_val.numpy(), y_val)

print("Best Params for AdversarialClassifier:", best_params_AdversarialClassifier, "with AUC:", best_auc_AdversarialClassifier)
print("Best Params for LinearClassifierAlexNet:", best_params_LinearClassifierAlexNet, "with AUC:", best_auc_LinearClassifierAlexNet)




0.5073529411764706
Best Params for AdversarialClassifier: {'nhid': 300, 'lr': 0.001, 'momentum': 0.93, 'num_epochs': 500, 'pca_components': 0.99} with AUC: 0.7720588235294118
Best Params for LinearClassifierAlexNet: {'n_hid': 100, 'lr': 0.01, 'momentum': 0.99, 'num_epochs': 500, 'pca_components': 0.9} with AUC: 0.6838235294117647


In [ ]:
# @title AUC
# @markdown let's try some more combinations. and also let's increase test set size ... hopefully that was the issue and not overfitting :)



#Increase reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

df = pd.read_csv('ADNI_Features_1-3v4.csv').dropna()
df = df[(df['Diagnosis'] == 1.0) | (df['Diagnosis'] == 3.0)] #leave out MCI but left in model training data for now
#df = df[df['source']!='train'] #take out model training data but leave val (not ideal)

from classifier import AdversarialClassifier
from classifier import LinearClassifierAlexNet

Y = df['Diagnosis']
X = df.drop(columns=['Diagnosis', 'scan_ID', 'source', 'SubjectID', 'SessionId', 'DiagnosisVisDate', 'UniqueID'])

scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

#map diagnoses
y_train = y_train.map({1.0: 0, 3.0: 1})
y_val = y_val.map({1.0: 0, 3.0: 1})
y_test = y_test.map({1.0: 0, 3.0: 1})


# Convert to pytorch tensors
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_val = torch.tensor(y_val.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)


#DataLoader for validation
val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(dataset=val_data, batch_size=64)



train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)


# Define parameter grids
param_grid_AdversarialClassifier = {
    #'nhid': [200, 300, 400, 500],
    'nhid': [400, 500],
    'lr': [0.001, 0.01],
    #'momentum': [0.9, 0.93, 0.95, 0.99],
    'momentum': [0.93, 0.99],
    'num_epochs' : [60, 500, 600], #500 epochs was best for both classifiers earlier, maybe higher would be even better?
    #'pca_components': [0.9, 0.95, 0.99, None]
    'pca_components': [0.95]
}

param_grid_LinearClassifierAlexNet = {
    #'n_hid': [200, 400, 500],
    'n_hid': [400, 500],
    'lr': [0.001, 0.0001],
    'momentum': [0.93, 0.99],
    'num_epochs' : [60, 500, 600],
    #'pca_components': [0.9, 0.95, 0.99, None]
    'pca_components': [0.95]
}

def apply_pca(X_train, X_val, n_components):
    if n_components is None:
        return X_train, X_val  # No PCA applied

    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_val_pca = pca.transform(X_val)  # Transform validation data using training PCA fit
    return X_train_pca, X_val_pca


def train_evaluate_model(model, train_loader, val_loader, num_epochs, lr, momentum):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                probabilities = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(probabilities)

                # Check for nans in predictions
                if np.isnan(probabilities).any():
                    raise ValueError("NaN found in model predictions")
    #and labels
    if np.isnan(all_labels).any():
        raise ValueError("NaN found in validation labels")

    auc_score = roc_auc_score(all_labels, all_preds)
    return auc_score


def grid_search(param_grid, classifier_type, X_train, y_train, X_val, y_val):
    best_auc = 0
    k = 0
    best_params = {}

    for params in itertools.product(*param_grid.values()):
        param_dict = dict(zip(param_grid.keys(), params))

        # Apply PCA w the number of components set
        X_train_pca, X_val_pca = apply_pca(X_train, X_val, param_dict['pca_components'])

        # need to adjust in_dim for the PCA-transformed data
        in_dim = X_train_pca.shape[1]

        # Make dataloaders for the PCA data
        train_data_pca = TensorDataset(torch.tensor(X_train_pca, dtype=torch.float32), y_train)
        train_loader_pca = DataLoader(dataset=train_data_pca, batch_size=4, shuffle=True) #batch size of 4 this time let's see if that's better
        val_data_pca = TensorDataset(torch.tensor(X_val_pca, dtype=torch.float32), y_val)
        val_loader_pca = DataLoader(dataset=val_data_pca, batch_size=4)

        # Initialize  model with the adjusted in_dim
        if classifier_type == 'AdversarialClassifier':
            model = AdversarialClassifier(in_dim=in_dim, nhid=param_dict['nhid'], out_dim=2)
        elif classifier_type == 'LinearClassifierAlexNet':
            model = LinearClassifierAlexNet(in_dim=in_dim, n_hid=param_dict['n_hid'], n_label=2)


        # Train and eval the model
        auc_score = train_evaluate_model(model, train_loader_pca, val_loader_pca, param_dict['num_epochs'], param_dict['lr'], param_dict['momentum'])
        k += 1
        print(auc_score, best_auc, k, param_dict)


        if auc_score > best_auc:
            best_auc = auc_score
            best_params = param_dict


    return best_params, best_auc


best_params_LinearClassifierAlexNet, best_auc_LinearClassifierAlexNet = grid_search(
    param_grid_LinearClassifierAlexNet, 'LinearClassifierAlexNet', X_train.numpy(), y_train, X_val.numpy(), y_val)

best_params_AdversarialClassifier, best_auc_AdversarialClassifier = grid_search(
    param_grid_AdversarialClassifier, 'AdversarialClassifier', X_train.numpy(), y_train, X_val.numpy(), y_val)


print("Best Params for AdversarialClassifier:", best_params_AdversarialClassifier, "with AUC:", best_auc_AdversarialClassifier)
print("Best Params for LinearClassifierAlexNet:", best_params_LinearClassifierAlexNet, "with AUC:", best_auc_LinearClassifierAlexNet)





In [ ]:
#Evaluate test set with best parameters based on grid search
seed = 19122
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Function to evaluate the model on the test set
def evaluate_test_set(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(probabilities)

    auc_score = roc_auc_score(all_labels, all_preds)
    return auc_score

# Apply PCA to the test set
X_train_pca, X_test_pca = apply_pca(X_train.numpy(), X_test.numpy(), best_params_AdversarialClassifier['pca_components'])

# Convert test set to PyTorch tensors
test_data = TensorDataset(torch.tensor(X_test_pca, dtype=torch.float32), y_test)
test_loader = DataLoader(dataset=test_data, batch_size=64)

# Initialize models with best parameters
best_model_AdversarialClassifier = AdversarialClassifier(in_dim=X_test_pca.shape[1], nhid=best_params_AdversarialClassifier['nhid'], out_dim=2)
best_model_LinearClassifierAlexNet = LinearClassifierAlexNet(in_dim=X_test_pca.shape[1], n_hid=best_params_LinearClassifierAlexNet['n_hid'], n_label=2)

# Evaluate AdversarialClassifier on test set
test_auc_AdversarialClassifier = evaluate_test_set(best_model_AdversarialClassifier, test_loader)
print("Test AUC for AdversarialClassifier:", test_auc_AdversarialClassifier)

# Evaluate LinearClassifierAlexNet on test set
test_auc_LinearClassifierAlexNet = evaluate_test_set(best_model_LinearClassifierAlexNet, test_loader)
print("Test AUC for LinearClassifierAlexNet:", test_auc_LinearClassifierAlexNet)


In [ ]:


# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load and preprocess your data here
# Assume df, X_train, X_test, y_train, y_test are already defined

# Function to apply PCA
def apply_pca(X_train, X_test, n_components):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca

# Use best params from the paper
best_params_AdversarialClassifier = {'nhid': 200, 'lr': 0.01, 'momentum': 0.9, 'num_epochs': 60, 'pca_components': 0.95}
best_params_LinearClassifierAlexNet = {'n_hid': 200, 'lr': 0.01, 'momentum': 0.9, 'num_epochs': 60, 'pca_components': 0.95}

# Apply PCA to the train and test set
X_train_pca, X_test_pca = apply_pca(X_train.numpy(), X_test.numpy(), best_params_AdversarialClassifier['pca_components'])

# train the model first
def train_model(model, train_loader, num_epochs, lr, momentum):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

# convert training data to tensors
train_data = TensorDataset(torch.tensor(X_train_pca, dtype=torch.float32), y_train)
train_loader = DataLoader(dataset=train_data, batch_size=64)

# Initialize and train adversarial classifier
best_model_AdversarialClassifier = AdversarialClassifier(in_dim=X_train_pca.shape[1], nhid=best_params_AdversarialClassifier['nhid'], out_dim=2)
train_model(best_model_AdversarialClassifier, train_loader, best_params_AdversarialClassifier['num_epochs'], best_params_AdversarialClassifier['lr'], best_params_AdversarialClassifier['momentum'])

# Initialize and train linear classifier
best_model_LinearClassifierAlexNet = LinearClassifierAlexNet(in_dim=X_train_pca.shape[1], n_hid=best_params_LinearClassifierAlexNet['n_hid'], n_label=2)
train_model(best_model_LinearClassifierAlexNet, train_loader, best_params_LinearClassifierAlexNet['num_epochs'], best_params_LinearClassifierAlexNet['lr'], best_params_LinearClassifierAlexNet['momentum'])

# evaluates the model on the test set
def evaluate_test_set(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(probabilities)
    auc_score = roc_auc_score(all_labels, all_preds)
    return auc_score

# Convert test set to PyTorch tensors
test_data = TensorDataset(torch.tensor(X_test_pca, dtype=torch.float32), y_test)
test_loader = DataLoader(dataset=test_data, batch_size=4)

# Evaluate adversarial classifier on test set
test_auc_AdversarialClassifier = evaluate_test_set(best_model_AdversarialClassifier, test_loader)
print("Test AUC for AdversarialClassifier:", test_auc_AdversarialClassifier)

# Evaluate linear classifier on test set
test_auc_LinearClassifierAlexNet = evaluate_test_set(best_model_LinearClassifierAlexNet, test_loader)
print("Test AUC for LinearClassifierAlexNet:", test_auc_LinearClassifierAlexNet)


Test AUC for AdversarialClassifier: 0.5582010582010581
Test AUC for LinearClassifierAlexNet: 0.5476190476190477
